In [1]:
import torch
import copy
import numpy as np
from tqdm import tqdm
import os
from utils.prune_utils.concern_identification import ConcernIdentification
from utils.prune_utils.weight_remover import WeightRemover
from utils.dataset import download_musdb, load_data
from utils.train import train_model, valid, load_not_compatible_weights, get_model_from_config
from utils.config_utils import load_config

In [2]:
torch.backends.cudnn.benchmark = True
torch.backends.cudnn.deterministic = False  # Fix possible slow down with dilation convolutions
torch.multiprocessing.set_start_method('spawn')
train_data, test_data = download_musdb()

In [3]:
device = 'cuda:0'
cfg = load_config('train_config.yaml')
cfg.model_type = 'htdemucs'
cfg.config_path = 'Configs/htdemucs_config.yaml'
cfg.results_path = 'Results/'
cfg.data_path = 'Datasets/musdb18hq/train'
cfg.num_workers = 0
cfg.valid_path = 'Datasets/musdb18hq/valid'
cfg.seed = 44
cfg.start_check_point = 'Results/model_htdemucs(before prune).ckpt'

In [4]:
htdemucs, htdemucs_config = get_model_from_config(cfg.model_type, cfg.config_path)
use_amp = htdemucs_config.training.get('use_amp', True)
load_not_compatible_weights(htdemucs, cfg.start_check_point, verbose=False)

train_dataloader, valid_dataloader, test_dataloader = load_data(htdemucs_config, cfg, 2)
htdemucs = htdemucs.to(device)
valid(htdemucs, cfg, htdemucs_config, device)
ref_htdemucs = copy.deepcopy(htdemucs)
weight_remover = WeightRemover(htdemucs, device, 0.8)

Use augmentation for training
Loading songs data from cache: Results/metadata_train.pkl. If you updated dataset remove metadata_train.pkl before training!
Found tracks in dataset: 80
Use augmentation for training
Loading songs data from cache: Results/metadata_valid.pkl. If you updated dataset remove metadata_valid.pkl before training!
Found tracks in dataset: 20
Use augmentation for training
Loading songs data from cache: Results/metadata_test.pkl. If you updated dataset remove metadata_test.pkl before training!
Found tracks in dataset: 50


100%|██████████| 20/20 [01:30<00:00,  4.54s/it, sdr_vocals=6.86, sdr_other=17.8]

Instr SDR vocals: 9.6452
Instr SDR other: 16.0977
SDR Avg: 12.8715


In [5]:
print("start pruning")
for idx in range(10):
    tr = torch.tensor(np.random.rand(2, 2, 485100), dtype=torch.float32).to(device)
    with torch.no_grad():
        y_ = weight_remover.process(tr)
    weight_remover.apply_removal()
valid(htdemucs, cfg, htdemucs_config, device)

start pruning
before 1048576
after 940206
before 1048576
after 829776
before 1048576
after 933425
before 1048576
after 938784
before 1048576
after 936106
before 1048576
after 811363
before 1048576
after 934726
before 1048576
after 937617
before 1048576
after 930812
before 1048576
after 894274
before 1048576
after 937072
before 1048576
after 939062
before 1048576
after 934443
before 1048576
after 908632
before 1048576
after 936549
before 1048576
after 936693
before 1048576
after 939562
before 1048576
after 940741
before 1048576
after 941313
before 1048576
after 936769
before 940206
after 938944
before 933425
after 932252
before 938784
after 936310
before 936106
after 933644
before 934726
after 933461
before 937617
after 935540
before 930812
after 928294
before 894274
after 890587
before 937072
after 936052
before 939062
after 937357
before 934443
after 931718
before 908632
after 905469
before 936549
after 935359
before 936693
after 935045
before 939562
after 937578
before 940741
after 9

100%|██████████| 20/20 [01:20<00:00,  4.02s/it, sdr_vocals=6.86, sdr_other=17.8]

Instr SDR vocals: 9.6483
Instr SDR other: 16.1008
SDR Avg: 12.8745


12.874510557480452

In [6]:
print("CI")
ci = ConcernIdentification(ref_htdemucs, htdemucs, device, 0.7)
_, temp_config = get_model_from_config(cfg.model_type, cfg.config_path)
temp_config.training.instruments = ["vocals"]
temp_dataloader, _, _ = load_data(temp_config, cfg, 2)

CI
Use augmentation for training
Loading songs data from cache: Results/metadata_train.pkl. If you updated dataset remove metadata_train.pkl before training!
Found tracks in dataset: 80
Use augmentation for training
Loading songs data from cache: Results/metadata_valid.pkl. If you updated dataset remove metadata_valid.pkl before training!
Found tracks in dataset: 20
Use augmentation for training
Loading songs data from cache: Results/metadata_test.pkl. If you updated dataset remove metadata_test.pkl before training!
Found tracks in dataset: 50


In [7]:
pbar = tqdm(temp_dataloader)
for i, (batch, mixes) in enumerate(pbar):
    y = batch.to(device)
    x = mixes.to(device)
    with torch.no_grad():
        ci.process(x)
    ci.apply_prune()
    if i > 100:
        break

  0%|          | 0/1000 [00:00<?, ?it/s]

before 938944
after 850138
before 829776
after 755656
before 932252
after 838039
before 936292
after 844179
before 933644
after 843798
before 811363
after 715547
before 933461
after 840500
before 935507
after 843991
before 928294
after 838018
before 890538
after 787637
before 936052
after 842421
before 937354
after 845635
before 931714


  0%|          | 1/1000 [00:01<26:41,  1.60s/it]

after 841764
before 905447
after 805713
before 935358
after 843004
before 935039
after 843867
before 937575
after 845341
before 938548
after 846115
before 940123
after 847430
before 934957
after 842442


  0%|          | 2/1000 [00:02<22:12,  1.34s/it]

...

before 444356
after 444356
before 506724
after 506070
before 426946
after 426946
before 428108
after 427515
before 468538
after 468538
before 512443
after 512060
before 527632
after 527554
before 473465
after 473465
before 466379
after 465819
before 467632
after 467632
before 562407
after 562390
before 537106
after 537106
before 442313
after 440967
before 505740
after 505629


 10%|█         | 101/1000 [01:32<13:15,  1.13it/s]

before 444356
after 444356
before 506070
after 506069
before 426946
after 426946
before 427515
after 427340
before 468538
after 468538
before 512060
after 512059
before 527554
after 527518
before 473465
after 473465
before 465819
after 465696
before 467632
after 467632
before 562390
after 562376
before 537106
after 537106
before 440967
after 440473
before 505629
after 505629


 10%|█         | 101/1000 [01:33<13:51,  1.08it/s]

before 444356
after 444356
before 506069
after 506069
before 426946
after 426946
before 427340
after 427173
before 468538
after 468538
before 512059
after 512059
before 527518
after 527489
before 473465
after 473465
before 465696
after 465147
before 467632
after 467632
before 562376
after 562310
before 537106
after 537106
before 440473
after 439595
before 505629
after 505629


In [8]:
store_path = os.path.join(cfg.results_path, f'model_htdemucs(after prune).ckpt')
state_dict = htdemucs.state_dict()
torch.save(state_dict, store_path)